## Introduction/Business Problem

### Problem Background
Canada is a popular location for exchange students, between 2014 and 2017 there was  an increase of 34% in international students. Most of these international students were in Toronto, Vancouver and Montreal. The continuous increasing rate of foreign students requires a higher housing amount.

Finding a place to live is usually a hard task for foreign students as they have to take into account many aspects such as living expenses, amenities, transportation, among others. 

### Problem Description
When a company is trying to build apartments for students in Toronto, is important to look for the right location, that will make students choose them. In order to do that is important to consider:
    - public transport
    - shopping and eating 
    - distance to universities

### Target Audience
Stakeholders of a construction project are interested on making a good investment that can allow them to choose the right neighborhood in Toronto in order to rent all the apartments in a short period of time.

### Data Description
The data that will be used for the problem is a list of postal codes in Toronto. This data will be get from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and turned into a dataframe for further analysis.

After getting the information from the webpage previously mentioned the Foursquare location data will be used to look for the neighborhood with better benefits.

In [1]:
# import necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

import requests

import geocoder # To get latitude and lonitude

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(r,"html.parser")
tab = soup.find('table')
tab.text
ro = tab.find_all('tr')    # get how many rows are in the table
nrows = len(ro)
nrows

181

In [3]:
title = tab.tr.text # extract header of table from webpage
title = title.split('\n')
h1 = title[1]
h2 = title[3]
h3 = title[5]
print("Header 1:", h1, "\nHeader 2:", h2, "\nHeader 3:",h3)

Header 1: Postal Code 
Header 2: Borough 
Header 3: Neighborhood


In [4]:
data = []
for i in range(1,nrows):
    data_PC = ro[i].text.split('\n')[1]   # Get Postal Code data
    data_B = ro[i].text.split('\n')[3]    # Get Borough data
    data_N = ro[i].text.split('\n')[5]    # Get Neighborhood  data
    data_N = data_N.replace('/',',')      # Get all neighborhoods separated by ,
    data_N = data_N.replace('Not assigned',data_B)  # Neighborhoods with Not asigned are change to the same name as the borough
    if data_B != 'Not assigned':          # Cells with not assigned boroughs are ignored
        data.append((data_PC, data_B, data_N))
         

In [5]:
df = pd.DataFrame(data, columns = [h1,h2,h3])    # Transform list into dataframe
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
print(df.shape)

(103, 3)


We can see that there are 103 different postal codes in Toronto, in order to reduce the amount of information only Toronto's boroughs are going to be consider.

### Methodology

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.


In order to continue with the analysis we need to get the latitude and longitude of each postal code.

In [7]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv") # Upload csv file 
coordinates.rename(columns={"Postal Code": h1}, inplace=True) 
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
Neighborhoods_Toronto = df.merge(coordinates, on=h1)   # Merge dataframes
Neighborhoods_Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
# Get latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [10]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Neighborhoods_Toronto['Latitude'], Neighborhoods_Toronto['Longitude'], Neighborhoods_Toronto['Borough'], Neighborhoods_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
# Get bouroughs in Toronto
r, c = Neighborhoods_Toronto.shape

N_T = []

for i in range(r):
    if "Toronto" in Neighborhoods_Toronto['Borough'][i]:
        if Neighborhoods_Toronto['Borough'][i] not in N_T:
            N_T.append(Neighborhoods_Toronto['Borough'][i])
N_T

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [12]:
# New dataframe of Toronto data
toronto_data = Neighborhoods_Toronto[Neighborhoods_Toronto['Borough'].isin(N_T)].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [13]:
toronto_data.shape

(39, 5)

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'HZKUXU35WHGBY0BS4YVNX413YGLUPTAIU0SW2NS1WHGVX5OV' # your Foursquare ID
CLIENT_SECRET = 'NN0XUKATFP1WJ5QTN5MAQ54EEG02NBVKO2XEFPQOH3P1HT4X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HZKUXU35WHGBY0BS4YVNX413YGLUPTAIU0SW2NS1WHGVX5OV
CLIENT_SECRET:NN0XUKATFP1WJ5QTN5MAQ54EEG02NBVKO2XEFPQOH3P1HT4X


In [16]:
# Explore neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(1602, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [19]:
# Verifying how many venues were returned for each neighborhood.
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
Business reply mail Processing Centre,17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [21]:
# Analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.0000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,Business reply mail Processing Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,Central Bay Street,0.016667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.016667,0.000000,0.000000,0.000000,0.0000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,Church and Wellesley,0.027397,0.013699,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,...,0.013699,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.0000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.028571,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000


In [23]:
# Printing each neighborhood along with the top 5 most common venues.
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04
3        Bakery  0.04
4   Cheese Shop  0.04


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.12
1       Nightclub  0.08
2  Breakfast Spot  0.08
3     Coffee Shop  0.08
4     Yoga Studio  0.04


----Business reply mail Processing Centre----
                  venue  freq
0                  Park  0.06
1         Garden Center  0.06
2  Fast Food Restaurant  0.06
3            Smoke Shop  0.06
4        Farmers Market  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4     Boat or Ferry  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.07
2                Café  0.05
3      

In [24]:
# Top 10 venues in each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Café,Restaurant,Gourmet Shop,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Gym / Fitness Center,Furniture / Home Store,Intersection,Italian Restaurant,Convenience Store,Bakery
2,Business reply mail Processing Centre,Smoke Shop,Pizza Place,Skate Park,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Rental Car Location,Plane,Harbor / Marina,Boat or Ferry,Coffee Shop,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Bar,Ice Cream Shop,Thai Restaurant,Burger Joint,Japanese Restaurant


In [25]:
# Clustering Neighborhoods into 5 clusters
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 4, 0, 0, 0, 0], dtype=int32)

In [26]:
# Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Breakfast Spot,Pub,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop,Restaurant,Theater
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Department Store,Creperie,Restaurant,Seafood Restaurant,Cosmetics Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Trail,Women's Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store


In [27]:
# Visualizing clusters
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [49]:
#### Cluster 0

In [28]:
toronto_d = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]
toronto_d

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Coffee Shop,Park,Bakery,Breakfast Spot,Pub,Café,Theater,Mexican Restaurant,Shoe Store,Restaurant
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
2,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Café,Bubble Tea Shop,Restaurant,Theater
3,St. James Town,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Department Store,Creperie,Restaurant,Seafood Restaurant,Cosmetics Shop
4,The Beaches,0,Health Food Store,Pub,Trail,Women's Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
5,Berczy Park,0,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Seafood Restaurant,Cheese Shop,Café,Restaurant,Gourmet Shop,Pub
6,Central Bay Street,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Bar,Ice Cream Shop,Thai Restaurant,Burger Joint,Japanese Restaurant
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Thai Restaurant,Clothing Store,Deli / Bodega,Hotel,Gym,American Restaurant,Sushi Restaurant
9,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Middle Eastern Restaurant,Bank,Bar,Pool,Bus Stop,Café,Supermarket,Music Venue
10,"Harbourfront East, Union Station, Toronto Islands",0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Sporting Goods Shop,Brewery,Scenic Lookout,Fried Chicken Joint,Italian Restaurant


#### Cluster 1

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,1,Garden,Home Service,Women's Store,Cupcake Shop,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 2 

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,2,Park,Swim School,Bus Line,Ethiopian Restaurant,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


#### Cluster 3

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,3,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run
29,"Moore Park, Summerhill East",3,Park,Gym,Trail,Tennis Court,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Diner
33,Rosedale,3,Park,Playground,Trail,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 4

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Christie,4,Grocery Store,Café,Park,Diner,Nightclub,Restaurant,Candy Store,Italian Restaurant,Baby Store,Coffee Shop


After verifying most common venues in each cluster, clusters 1, 2, 3 and 4 are no longer an option for the departments. As the University of Toronto is the one with higher amount of students the distance from each neighborhood to the University is going to be computed.

In [34]:
address = 'University of Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_ut = location.latitude
longitude_ut = location.longitude
print('The geograpical coordinate of the University of Toronto are {}, {}.'.format(latitude_ut, longitude_ut))

The geograpical coordinate of the University of Toronto are 43.663461999999996, -79.39775965337452.


In [35]:
from math import sin, cos, sqrt, atan2, radians

def distance_u(lat1,lon1,lat2,lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon2)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance


In [36]:
toronto_new = toronto_merged[toronto_merged['Cluster Labels'] == 0]

In [37]:
lat = toronto_new['Latitude']
lon = toronto_new['Longitude']

In [38]:
lat = lat.tolist()
lon = lon.tolist()

In [39]:
dist_neigh = []
for i in range(33):
    lat2 = lat[i]
    lon2 = lon[i]
    dist = distance_u(latitude_ut,longitude_ut,lat2,lon2)
    dist_neigh.append(dist)



In [40]:
toronto_new.insert(3,'Distance to UT', dist_neigh)

In [47]:
toronto_new.sort_values(by ='Distance to UT',ascending=True)

,Postal Code,Borough,Neighborhood,Distance to UT,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,M7Y,East Toronto,Business reply mail Processing Centre,0.079874,43.662744,-79.321558,0,Smoke Shop,Pizza Place,Skate Park,Brewery,Burrito Place,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio,Butcher
27,M5S,Downtown Toronto,"University of Toronto, Harbord",0.085247,43.662696,-79.400049,0,Café,Bookstore,Italian Restaurant,Bar,Japanese Restaurant,Restaurant,Bakery,Chinese Restaurant,Pub,Sushi Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.129082,43.662301,-79.389494,0,Coffee Shop,College Cafeteria,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
22,M6P,West Toronto,"High Park, The Junction South",0.206187,43.661608,-79.464763,0,Mexican Restaurant,Café,Thai Restaurant,Diner,Fast Food Restaurant,Italian Restaurant,Arts & Crafts Store,Cajun / Creole Restaurant,Fried Chicken Joint,Bar
37,M4Y,Downtown Toronto,Church and Wellesley,0.266718,43.665860,-79.383160,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Yoga Studio,Gay Bar,Mediterranean Restaurant,Café,Pub,Men's Store
17,M4M,East Toronto,Studio District,0.437856,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,American Restaurant,Brewery,Bakery,Latin American Restaurant,Pet Store,Bar,Seafood Restaurant
35,M4X,Downtown Toronto,"St. James Town, Cabbagetown",0.501090,43.667967,-79.367675,0,Bakery,Coffee Shop,Chinese Restaurant,Italian Restaurant,Pub,Café,Pizza Place,Park,Restaurant,Gift Shop
6,M5G,Downtown Toronto,Central Bay Street,0.612832,43.657952,-79.387383,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Bubble Tea Shop,Bar,Ice Cream Shop,Thai Restaurant,Burger Joint,Japanese Restaurant
15,M4L,East Toronto,"India Bazaar, The Beaches West",0.615824,43.668999,-79.315572,0,Park,Fast Food Restaurant,Food & Drink Shop,Sandwich Place,Brewery,Board Shop,Liquor Store,Italian Restaurant,Restaurant,Burrito Place
9,M6H,West Toronto,"Dufferin, Dovercourt Village",0.616558,43.669005,-79.442259,0,Pharmacy,Bakery,Middle Eastern Restaurant,Bank,Bar,Pool,Bus Stop,Café,Supermarket,Music Venue


## Results

1. The cluster with more venues of interest for students is cluster 0.
2. Only cluster 2 has "Bus line" as most common venues.
3. The nearest neighborhood to University of Toronto in cluster 0 is Business reply mail Processing Centre.
3. The furthest neighborhood to University of Toronto in cluster 0 is North Toronto West.

## Discussion and conclusion

As previously shown the best place to build departments for students is in cluster 0, then if you had to choose a specific neighborhood the top three according to distance and venues would be:
- University of Toronto, Harbord
- Regent Park, Harbourfront
- St. James Town, Cabbagetown